In [3]:
from datasets import load_dataset

ds = load_dataset("uoft-cs/cifar10", download_mode='reuse_cache_if_exists')

c:\Users\X\Anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 148923.77 examples/s]


In [22]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [5]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})


In [30]:
X_train = ds['train']['img']
Y_train = ds['train']['label']
X_test = ds['test']['img']
Y_test = ds['test']['label']

In [31]:
type(Y_test)

list

In [42]:
X_train = torch.from_numpy(np.array(X_train)).transpose(1, 3).float()
Y_train = torch.tensor(Y_train)
X_test = torch.from_numpy(np.array(X_test)).transpose(1, 3).float()
Y_test = torch.tensor(Y_test)

C:\Users\X\AppData\Local\Temp\ipykernel_25392\1783767769.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_train = torch.tensor(Y_train)
C:\Users\X\AppData\Local\Temp\ipykernel_25392\1783767769.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = torch.tensor(Y_test)


In [40]:
a = X_train.transpose(1, 3)

In [41]:
a.shape

torch.Size([50000, 3, 32, 32])

In [68]:
m0 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1, stride=1)
a1 = m0(a[:1])
print(a1.shape)
m1 = nn.MaxPool2d(2)
a2 = m1(a1)
print(a2.shape)

torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 16, 16])


c:\Users\X\Anaconda3\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1041.)
  return F.conv2d(input, weight, bias, self.stride,


In [69]:
class CNN(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        # in shape 3*32*32
        self.l1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1), # 32*29*29
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #32*14*14
            nn.BatchNorm2d(32)
        )
        # in shape 32*14*14
        self.l2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=4, stride=1), #32*11*11
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #32*5*5
            nn.BatchNorm2d(32),
            nn.Flatten()
        )
        # in shape 32*5*5
        self.l3 = nn.Sequential(
            nn.Linear(in_features=32*5*5, out_features=128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x
model = CNN()

In [71]:
out = model(X_train[:10])